In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_path = "/content/drive/MyDrive/Data"
train = "/content/drive/MyDrive/Data/train"
valid = "/content/drive/MyDrive/Data/valid"
test = "/content/drive/MyDrive/Data/test"

train_images = [f for f in os.listdir(train) if f.endswith(('.jpg'))]
train_labels = [f for f in os.listdir(train) if f.endswith('.txt')]

test_images = [f for f in os.listdir(test) if f.endswith(('.jpg'))]
test_labels = [f for f in os.listdir(test) if f.endswith('.txt')]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2

next_train_images = [os.path.join(train, fname) for fname in train_images[pic_index-2:pic_index]]
next_test_images = [os.path.join(test, fname) for fname in test_images[pic_index-2:pic_index]]

for i, img_path in enumerate(next_train_images + next_test_images):
    img = mpimg.imread(img_path)
    plt.imshow(img)
    plt.axis('Off')
    plt.show()

print(len(train_images), len(test_images))

In [ ]:
# 이미지 리사이즈 함수
def resize_images_train(image_paths, label_paths, target_size=(150, 150)):
    resized_images = []
    updated_labels = []
    for img_path in image_paths:
        img_full_path = os.path.join(train, img_path)
        img = cv2.imread(img_full_path)  
        
        if img is None:
            print(f"Warning: Unable to read image at {img_full_path}")
            continue

    for label_path in label_paths:
        label_full_path = os.path.join(train, label_path)
        with open(label_full_path, 'r') as file:
            lines = file.readlines()
        
        img_resized = cv2.resize(img, target_size)  # 이미지 리사이즈
        resized_images.append(img_resized)

        new_label_lines = []
        for line in lines:
            class_id, x_center, y_center, width, height = map(float, line.split())
            
            # 리사이즈된 이미지의 새로운 좌표 계산
            new_x_center = x_center * (target_size[0] / img.shape[1])
            new_y_center = y_center * (target_size[1] / img.shape[0])
            new_width = width * (target_size[0] / img.shape[1])
            new_height = height * (target_size[1] / img.shape[0])
            
            new_label_lines.append(f"{int(class_id)} {new_x_center} {new_y_center} {new_width} {new_height}\n")

        # 업데이트된 라벨 저장
        updated_labels.append(new_label_lines)

    return resized_images, updated_labels

def resize_images_test(image_paths, label_paths, target_size=(150, 150)):
    resized_images = []
    updated_labels = []
    for img_path in image_paths:
        img_full_path = os.path.join(test, img_path) 
        img = cv2.imread(img_full_path)  

        if img is None:
            print(f"Warning: Unable to read image at {img_full_path}")
            continue 

    for label_path in label_paths:
        label_full_path = os.path.join(test, label_path)
        with open(label_full_path, 'r') as file:
            lines = file.readlines()
        
        img_resized = cv2.resize(img, target_size)  # 이미지 리사이즈
        resized_images.append(img_resized)

        new_label_lines = []
        for line in lines:
            class_id, x_center, y_center, width, height = map(float, line.split())
            
            # 리사이즈된 이미지의 새로운 좌표 계산
            new_x_center = x_center * (target_size[0] / img.shape[1])
            new_y_center = y_center * (target_size[1] / img.shape[0])
            new_width = width * (target_size[0] / img.shape[1])
            new_height = height * (target_size[1] / img.shape[0])
            
            new_label_lines.append(f"{int(class_id)} {new_x_center} {new_y_center} {new_width} {new_height}\n")

        # 업데이트된 라벨 저장
        updated_labels.append(new_label_lines)

    return resized_images, updated_labels


# 이미지 리사이즈 적용
resized_train_images, updated_train_labels = resize_images_train(train_images, train_labels)
resized_test_images, updated_test_labels = resize_images_test(test_images, test_labels)


print("Resized Train Images:", len(resized_train_images))
print("Updated Train Labels:", len(updated_train_labels))
print("Resized Test Images:", len(resized_test_images))
print("Updated Test Labels:", len(updated_test_labels))

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    verbose=True
)

In [ ]:
model.save("model_cnn.h5")

model = keras.models.load_model("model_cnn.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

tflite_model_path = "model_cnn.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print("TFLite model saved to", tflite_model_path)

os.system(f"edgetpu_compiler {tflite_model_path}")

print("Edge TPU model compiled.")